# Подсчет числа правильных ответов GSM8K

In [1]:
str_to_delete = '''prompt="\nProblem:\nOlivia has $23. She bought five cupcakes for $3 each and a milkshake for $4. How much money does she have left?\nSolution:\nFirst, calculate the total cost of the cupcakes:\n5 cupcakes × $3 = $15.\nAdd the cost of the milkshake: $15 + $4 = $19.\nSubtract from her initial amount: $23 − $19 = $4.\nSo, Olivia has $4 left.\nFinal Answer: 4\u200b\n\nProblem:\nA bakery sells cookies in packs of 6. If a customer buys 9 packs, how many cookies does the customer get in total?\nSolution:\nEach pack contains 6 cookies.\nThe customer buys 9 packs.\nTotal cookies = 6 × 9 = 54.\nFinal Answer: 54\n\nProblem:\nThere are 42 students in a class. One-third of them are boys. How many girls are in the class?\nSolution:\nNumber of boys = 42 ÷ 3 = 14.\nNumber of girls = total students − boys = 42 − 14 = 28.\nFinal Answer: 28\n\nProblem:\nA car travels 60 miles per hour. How many miles does it travel in 2 hours and 30 minutes?\nSolution:\nConvert 2 hours 30 minutes to hours: 2.5 hours.\nDistance = speed × time = 60 × 2.5 = 150 miles.\nFinal Answer: 150\n\nProblem:\nJames has 3 times as many marbles as Lisa. Together, they have 48 marbles. How many marbles does James have?\n\nSolution:\nLet Lisa have x marbles.\nThen James has 3x marbles.\nTogether: x + 3x = 4x = 48.\nSo, x = 48 ÷ 4 = 12.\nJames has 3 × 12 = 36 marbles.\nFinal Answer: 36\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?'''

student_score = []
teacher_score = []

import glob
import re
txt_arr = []

for filename in glob.glob("*_nt_16.txt"):
    if '_nt' in filename:
        txt_arr.append(filename)

print()

for txt in txt_arr:
    print(f'{txt=}')
    with open(f'{txt}', 'r', encoding='utf-8') as f:
        data = f.read()
    if str_to_delete in data:
        data = data.replace(str_to_delete, '')
    splitted = data.split(f'-----'*20)
    # print(splitted)

    comparison_dict = {'student_score': 0,
                       'teacher_score': 0}

    for task in splitted:
        # Correct answer
        correct_answer = task[task.find('Correct answer: ')+len('Correct answer: '):task.find('\n\nStudent solution')]
        correct_answer = correct_answer.replace(',', '').replace('.', '')
        
        # Student parsed answer
        idx = task.find('\n\nTeacher solution')
        if idx == -1:
            student_answer = task[-50:]
        else:
            start = max(0, idx - 50)
            student_answer = task[start:idx]

        student_answer = student_answer.replace(',', '').replace('.', '')

        # Teacher parsed answer
        teacher_answer = task[-50:].replace(',', '').replace('.', '')

        student_answer = re.findall(r'-?\b\d+\b', student_answer)
        teacher_answer = re.findall(r'-?\b\d+\b', teacher_answer)
        
        # print(f"{correct_answer=}")
        # print(f"{student_answer=}")
        # print(f"{teacher_answer=}")

        if correct_answer in student_answer:
            comparison_dict['student_score'] += 1
        if correct_answer in teacher_answer:
            comparison_dict['teacher_score'] += 1
        
        # print('---------------------------------')
    print(f'{comparison_dict=}')
    student_score.append(comparison_dict['student_score'])
    teacher_score.append(comparison_dict['teacher_score'])
    print('---------------------------------')

# Подсчет числа правильных ответов CSQA

In [2]:
str_to_delete = '''prompt="\nProblem:\nOlivia has $23. She bought five cupcakes for $3 each and a milkshake for $4. How much money does she have left?\nSolution:\nFirst, calculate the total cost of the cupcakes:\n5 cupcakes × $3 = $15.\nAdd the cost of the milkshake: $15 + $4 = $19.\nSubtract from her initial amount: $23 − $19 = $4.\nSo, Olivia has $4 left.\nFinal Answer: 4\u200b\n\nProblem:\nA bakery sells cookies in packs of 6. If a customer buys 9 packs, how many cookies does the customer get in total?\nSolution:\nEach pack contains 6 cookies.\nThe customer buys 9 packs.\nTotal cookies = 6 × 9 = 54.\nFinal Answer: 54\n\nProblem:\nThere are 42 students in a class. One-third of them are boys. How many girls are in the class?\nSolution:\nNumber of boys = 42 ÷ 3 = 14.\nNumber of girls = total students − boys = 42 − 14 = 28.\nFinal Answer: 28\n\nProblem:\nA car travels 60 miles per hour. How many miles does it travel in 2 hours and 30 minutes?\nSolution:\nConvert 2 hours 30 minutes to hours: 2.5 hours.\nDistance = speed × time = 60 × 2.5 = 150 miles.\nFinal Answer: 150\n\nProblem:\nJames has 3 times as many marbles as Lisa. Together, they have 48 marbles. How many marbles does James have?\n\nSolution:\nLet Lisa have x marbles.\nThen James has 3x marbles.\nTogether: x + 3x = 4x = 48.\nSo, x = 48 ÷ 4 = 12.\nJames has 3 × 12 = 36 marbles.\nFinal Answer: 36\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?'''

student_score = []
teacher_score = []

import glob
import re
student_arr = []
teacher_arr = []

student_arr = glob.glob("./student_outputs_CSQA_posttrain/*_16.txt")
teacher_arr = glob.glob("./student_outputs_CSQA_posttrain/*_16.txt")


print(teacher_arr)

['./student_outputs_CSQA_posttrain/3_16.txt', './student_outputs_CSQA_posttrain/1_16.txt', './student_outputs_CSQA_posttrain/10_16.txt', './student_outputs_CSQA_posttrain/9_16.txt', './student_outputs_CSQA_posttrain/5_16.txt', './student_outputs_CSQA_posttrain/7_16.txt', './student_outputs_CSQA_posttrain/2_16.txt', './student_outputs_CSQA_posttrain/8_16.txt', './student_outputs_CSQA_posttrain/4_16.txt', './student_outputs_CSQA_posttrain/6_16.txt']


In [3]:
for txt in student_arr:
    print(f'{txt=}')
    with open(f'{txt}', 'r', encoding='utf-8') as f:
        data = f.read()
    if str_to_delete in data:
        data = data.replace(str_to_delete, '')
    splitted = data.split(f'-----'*20)
    comparison_dict = {'score': 0}

    for task in splitted:
        # Correct answer
        correct_answer = task[task.find('Correct answer: ')+len('Correct answer: '):task.find('\n\nStudent solution')]
        
        # Student parsed answer
        if 'is: **' in task:
            student_answer = task[task.find('is: **')+len('is: **'):task.find(' —')]
        elif 'is **' in task:
            student_answer = task[task.find('is **')+len('is **'):task.find(' —')]

        if correct_answer == student_answer:
            comparison_dict['score'] += 1

        # print('---------------------------------')
    print(f'{comparison_dict=}')
    student_score.append(comparison_dict['score'])
    print('---------------------------------')

txt='./student_outputs_CSQA_posttrain/3_16.txt'
comparison_dict={'score': 50}
---------------------------------
txt='./student_outputs_CSQA_posttrain/1_16.txt'
comparison_dict={'score': 51}
---------------------------------
txt='./student_outputs_CSQA_posttrain/10_16.txt'
comparison_dict={'score': 56}
---------------------------------
txt='./student_outputs_CSQA_posttrain/9_16.txt'
comparison_dict={'score': 50}
---------------------------------
txt='./student_outputs_CSQA_posttrain/5_16.txt'
comparison_dict={'score': 49}
---------------------------------
txt='./student_outputs_CSQA_posttrain/7_16.txt'
comparison_dict={'score': 52}
---------------------------------
txt='./student_outputs_CSQA_posttrain/2_16.txt'
comparison_dict={'score': 52}
---------------------------------
txt='./student_outputs_CSQA_posttrain/8_16.txt'
comparison_dict={'score': 48}
---------------------------------
txt='./student_outputs_CSQA_posttrain/4_16.txt'
comparison_dict={'score': 54}
-------------------------

In [4]:
sum(student_score)/len(student_score)

51.3

In [5]:
for txt in teacher_arr:
    print(f'{txt=}')
    with open(f'{txt}', 'r', encoding='utf-8') as f:
        data = f.read()
    if str_to_delete in data:
        data = data.replace(str_to_delete, '')
    splitted = data.split(f'-----'*20)
    comparison_dict = {'score': 0}

    for task in splitted:
        # Correct answer
        correct_answer = task[task.find('Correct answer: ')+len('Correct answer: '):task.find('\n\nStudent solution')]
        lower_case = task.lower()
        # print(task)
        # Student parsed answer

        # Ищем шаблон вроде **A**, **B — text**, **C. something**
        import re

        # Извлекаем correct_answer как раньше (но аккуратно)
        # correct_part = task.split('Correct answer: ', 1)[-1].split('\n\nStudent solution')[0].strip()
        # correct_answer = correct_part.split()[0]  # Берём первую лексему: "A", "B", и т.д.

        # Для студента: ищем в конце Student solution последнюю строку с "answer"
        student_text = task.split('Student solution:', 1)[-1]

        # Ищем шаблоны типа:
        # - "Correct answer: A"
        # - "Final Answer: **B**"
        # - "So the best answer is: C"
        # - "Answer: D — ..."
        # - "✅ Correct Answer: E"

        patterns = [
            r'(?:correct|final|best)?\s*answer\s*[—:-]?\s*\*?([A-E])\*?',
            r'answer\s+is\s*:?\s*\*?([A-E])\*?',
            r'so the (?:best|correct) answer is\s*:?\s*\*?([A-E])',
            r'^\s*\*?([A-E])\s*—',
            r'\b([A-E])\s*—\s*[a-z]',
        ]

        student_answer = None
        for pattern in patterns:
            match = re.search(pattern, student_text, re.IGNORECASE | re.MULTILINE)
            if match:
                student_answer = match.group(1).lower()
                break

        # Если не нашли — пробуем найти последнюю заглавную букву A–E перед тире или в конце
        if student_answer is None:
            # Ищем все A-E в тексте и берём последнюю (часто работает)
            candidates = re.findall(r'\b([A-E])\b', student_text)
            if candidates:
                student_answer = candidates[-1].lower()

        # if student_answer is None:
        #     print("Не удалось распарсить ответ студента")
        # else:
        #     print(f"Извлечён ответ: {student_answer}")

        # if 'answer is: **' in lower_case:
        #     student_answer = lower_case[lower_case.find('answer is: **')+len('answer is: **'):lower_case.find(' —')]
        #     print(f'попали 1, распарсили {student_answer}')
        # elif 'answer: **' in lower_case:
        #     student_answer = lower_case[lower_case.find('answer: **')+len('answer: **'):lower_case.find(' —')]
        #     print(f'попали 2, распарсили {student_answer}')
        # elif 'answer is:\n\n**' in lower_case.lower():
        #     student_answer = lower_case[lower_case.find('answer is:\n\n**')+len('answer is:\n\n**'):lower_case.find(' —')]
        #     print(f'попали 3, распарсили {student_answer}')
        # elif 'answer is:\n**' in lower_case.lower():
        #     student_answer = lower_case[lower_case.find('answer is:\n**')+len('answer is:\n**'):lower_case.find(' —')]
        #     print(f'попали 4, распарсили {student_answer}')
        # elif 'answer : **' in lower_case.lower():
        #     student_answer = lower_case[lower_case.find('answer : **')+len('answer : **'):lower_case.find(' —')]
        #     print(f'попали 5, распарсили {student_answer}')

        if correct_answer.lower() == student_answer:
            comparison_dict['score'] += 1

        # print('---------------------------------')
    print(f'{comparison_dict=}')
    teacher_score.append(comparison_dict['score'])
    print('---------------------------------')

txt='./student_outputs_CSQA_posttrain/3_16.txt'
comparison_dict={'score': 53}
---------------------------------
txt='./student_outputs_CSQA_posttrain/1_16.txt'
comparison_dict={'score': 52}
---------------------------------
txt='./student_outputs_CSQA_posttrain/10_16.txt'
comparison_dict={'score': 57}
---------------------------------
txt='./student_outputs_CSQA_posttrain/9_16.txt'
comparison_dict={'score': 49}
---------------------------------
txt='./student_outputs_CSQA_posttrain/5_16.txt'
comparison_dict={'score': 51}
---------------------------------
txt='./student_outputs_CSQA_posttrain/7_16.txt'
comparison_dict={'score': 51}
---------------------------------
txt='./student_outputs_CSQA_posttrain/2_16.txt'
comparison_dict={'score': 52}
---------------------------------
txt='./student_outputs_CSQA_posttrain/8_16.txt'
comparison_dict={'score': 51}
---------------------------------
txt='./student_outputs_CSQA_posttrain/4_16.txt'
comparison_dict={'score': 55}
-------------------------

In [7]:
sum(teacher_score)/len(teacher_score)

52.2

# Подсчет числа правильных ответов Coin Flip

In [22]:
str_to_delete = '''prompt="\nProblem:\nOlivia has $23. She bought five cupcakes for $3 each and a milkshake for $4. How much money does she have left?\nSolution:\nFirst, calculate the total cost of the cupcakes:\n5 cupcakes × $3 = $15.\nAdd the cost of the milkshake: $15 + $4 = $19.\nSubtract from her initial amount: $23 − $19 = $4.\nSo, Olivia has $4 left.\nFinal Answer: 4\u200b\n\nProblem:\nA bakery sells cookies in packs of 6. If a customer buys 9 packs, how many cookies does the customer get in total?\nSolution:\nEach pack contains 6 cookies.\nThe customer buys 9 packs.\nTotal cookies = 6 × 9 = 54.\nFinal Answer: 54\n\nProblem:\nThere are 42 students in a class. One-third of them are boys. How many girls are in the class?\nSolution:\nNumber of boys = 42 ÷ 3 = 14.\nNumber of girls = total students − boys = 42 − 14 = 28.\nFinal Answer: 28\n\nProblem:\nA car travels 60 miles per hour. How many miles does it travel in 2 hours and 30 minutes?\nSolution:\nConvert 2 hours 30 minutes to hours: 2.5 hours.\nDistance = speed × time = 60 × 2.5 = 150 miles.\nFinal Answer: 150\n\nProblem:\nJames has 3 times as many marbles as Lisa. Together, they have 48 marbles. How many marbles does James have?\n\nSolution:\nLet Lisa have x marbles.\nThen James has 3x marbles.\nTogether: x + 3x = 4x = 48.\nSo, x = 48 ÷ 4 = 12.\nJames has 3 × 12 = 36 marbles.\nFinal Answer: 36\n\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?'''

student_score = []
teacher_score = []

import glob
import re
student_arr = []
teacher_arr = []

student_arr = glob.glob("./student_outputs_Coin_Flip_posttrain/*_16.txt")
teacher_arr = glob.glob("./teacher_outputs_Coin_flip_pretrain/*_16.txt")


print(student_arr)

['./student_outputs_Coin_Flip_posttrain/3_16.txt', './student_outputs_Coin_Flip_posttrain/1_16.txt', './student_outputs_Coin_Flip_posttrain/10_16.txt', './student_outputs_Coin_Flip_posttrain/9_16.txt', './student_outputs_Coin_Flip_posttrain/5_16.txt', './student_outputs_Coin_Flip_posttrain/7_16.txt', './student_outputs_Coin_Flip_posttrain/2_16.txt', './student_outputs_Coin_Flip_posttrain/8_16.txt', './student_outputs_Coin_Flip_posttrain/4_16.txt', './student_outputs_Coin_Flip_posttrain/6_16.txt']


In [23]:
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [24]:
p_arr = []
r_arr = []
f1_arr = []

for txt in student_arr:
    print(f'{txt=}')
    with open(f'{txt}', 'r', encoding='utf-8') as f:
        data = f.read()
    if str_to_delete in data:
        data = data.replace(str_to_delete, '')
    splitted = data.split(f'-----'*20)
    comparison_dict = {'score': 0}

    llm_answers = []
    corr_answers = []
    for task in splitted:
        # Correct answer
        correct_answer = task[task.find('Correct answer: ')+len('Correct answer: '):task.find('\n\nStudent solution')]
        lower_case = task.lower()
        student_solution = lower_case[lower_case.find('student solution: ')+len('student solution: '):]     
        student_answer = student_solution[student_solution.find('**answer: ')+len('**answer: '):]
        if 'yes' in student_answer:
            llm_answers.append(1)
        else:
            llm_answers.append(0)

        if 'yes' in correct_answer:
            corr_answers.append(1)
        else:
            corr_answers.append(0)

    p = precision_score(corr_answers, llm_answers)
    r = recall_score(corr_answers, llm_answers)
    f1 = f1_score(corr_answers, llm_answers)
    p_arr.append(p)
    r_arr.append(r)
    f1_arr.append(f1)
    print(f'{p=}')
    print(f'{r=}')
    print(f'{f1=}')
    print('---------------------------------')

txt='./student_outputs_Coin_Flip_posttrain/3_16.txt'
p=0.9545454545454546
r=0.9130434782608695
f1=0.9333333333333333
---------------------------------
txt='./student_outputs_Coin_Flip_posttrain/1_16.txt'
p=1.0
r=0.9183673469387755
f1=0.9574468085106383
---------------------------------
txt='./student_outputs_Coin_Flip_posttrain/10_16.txt'
p=0.9545454545454546
r=0.8936170212765957
f1=0.9230769230769231
---------------------------------
txt='./student_outputs_Coin_Flip_posttrain/9_16.txt'
p=0.9791666666666666
r=0.9791666666666666
f1=0.9791666666666666
---------------------------------
txt='./student_outputs_Coin_Flip_posttrain/5_16.txt'
p=0.9777777777777777
r=0.9166666666666666
f1=0.946236559139785
---------------------------------
txt='./student_outputs_Coin_Flip_posttrain/7_16.txt'
p=1.0
r=0.9375
f1=0.967741935483871
---------------------------------
txt='./student_outputs_Coin_Flip_posttrain/2_16.txt'
p=1.0
r=0.9591836734693877
f1=0.9791666666666666
---------------------------------
t

In [25]:
p_mean = sum(p_arr)/len(p_arr)
r_mean = sum(r_arr)/len(r_arr)
f1_mean = sum(f1_arr)/len(f1_arr)

print(f'{p_mean=}')
print(f'{r_mean=}')
print(f'{f1_mean=}')

p_mean=0.9818416305916307
r_mean=0.927966069252246
f1_mean=0.9539988082573816


In [8]:
p_arr = []
r_arr = []
f1_arr = []

for txt in teacher_arr:
    print(f'{txt=}')
    with open(f'{txt}', 'r', encoding='utf-8') as f:
        data = f.read()
    if str_to_delete in data:
        data = data.replace(str_to_delete, '')
    splitted = data.split(f'-----'*20)
    comparison_dict = {'score': 0}

    llm_answers = []
    corr_answers = []
    for task in splitted:
        # Correct answer
        correct_answer = task[task.find('Correct answer: ')+len('Correct answer: '):task.find('\n\nStudent solution')]
        lower_case = task.lower()
        student_solution = lower_case[lower_case.find('student solution: ')+len('student solution: '):]     
        student_answer = student_solution[student_solution.find('**answer: ')+len('**answer: '):]
        if 'yes' in student_answer:
            llm_answers.append(1)
        else:
            llm_answers.append(0)

        if 'yes' in correct_answer:
            corr_answers.append(1)
        else:
            corr_answers.append(0)

    p = precision_score(corr_answers, llm_answers)
    r = recall_score(corr_answers, llm_answers)
    f1 = f1_score(corr_answers, llm_answers)
    p_arr.append(p)
    r_arr.append(r)
    f1_arr.append(f1)
    print(f'{p=}')
    print(f'{r=}')
    print(f'{f1=}')
    print('---------------------------------')

txt='./teacher_outputs_Coin_flip_pretrain/3_16.txt'
p=0.9259259259259259
r=0.9259259259259259
f1=0.9259259259259259
---------------------------------
txt='./teacher_outputs_Coin_flip_pretrain/1_16.txt'
p=0.8846153846153846
r=1.0
f1=0.9387755102040817
---------------------------------
txt='./teacher_outputs_Coin_flip_pretrain/10_16.txt'
p=1.0
r=0.9642857142857143
f1=0.9818181818181818
---------------------------------
txt='./teacher_outputs_Coin_flip_pretrain/9_16.txt'
p=0.9642857142857143
r=1.0
f1=0.9818181818181818
---------------------------------
txt='./teacher_outputs_Coin_flip_pretrain/5_16.txt'
p=0.9629629629629629
r=1.0
f1=0.9811320754716981
---------------------------------
txt='./teacher_outputs_Coin_flip_pretrain/7_16.txt'
p=0.9259259259259259
r=0.8928571428571429
f1=0.9090909090909091
---------------------------------
txt='./teacher_outputs_Coin_flip_pretrain/2_16.txt'
p=0.967741935483871
r=1.0
f1=0.9836065573770492
---------------------------------
txt='./teacher_outputs_Co

In [9]:
p_mean = sum(p_arr)/len(p_arr)
r_mean = sum(r_arr)/len(r_arr)
f1_mean = sum(f1_arr)/len(f1_arr)

print(f'{p_mean=}')
print(f'{r_mean=}')
print(f'{f1_mean=}')

p_mean=0.9395146848372654
r_mean=0.9707570207570206
f1_mean=0.9542653171665542
